# Purpose of the notebook is to get a feel for the AutoGen libary and it's use cases 

In [1]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)

In [3]:
# create an AssistantAgent named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "seed": 42,  # seed for caching and reproducibility
        "config_list": config_list,  # a list of OpenAI API configurations
        "temperature": 0,  # temperature for sampling
    },  # configuration for autogen's enhanced inference API which is compatible with OpenAI API
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="ALWAYS",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,  # set to True or image name like "python:3" to use docker
    },
)
# the assistant receives a message from the user_proxy, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""What date is today? Compare the year-to-date gain for META and TESLA. Give me a visualziation plot""",
)

user_proxy (to assistant):

What date is today? Compare the year-to-date gain for META and TESLA. Give me a visualziation plot

--------------------------------------------------------------------------------
assistant (to user_proxy):

First, let's get the current date. We can do this using Python's built-in datetime module. Here is the Python code to get the current date:

```python
# python code
import datetime

# Get the current date
current_date = datetime.date.today()

print("Today's date is:", current_date)
```

Next, we need to get the year-to-date (YTD) gain for META (Facebook) and TESLA. We can do this by using the yfinance module in Python, which allows us to download stock data from Yahoo Finance. Here is the Python code to get the YTD gain for META and TESLA:

```python
# python code
import yfinance as yf

# Get the data for the stock META and TESLA
meta = yf.Ticker("META")
tesla = yf.Ticker("TSLA")

# Get the historical market data
meta_hist = meta.history(period="ytd")
t

user_proxy (to assistant):

exitcode: 1 (execution failed)
Code output: 
FB: No data found, symbol may be delisted
:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  meta_ytd_gain = (meta_hist['Close'][-1] - meta_hist['Close'][0]) / meta_hist['Close'][0]
Traceback (most recent call last):
  File "", line 20, in <module>
    meta_ytd_gain = (meta_hist['Close'][-1] - meta_hist['Close'][0]) / meta_hist['Close'][0]
  File "c:\Users\stu-kalebnim\Documents\GitHub\Mointor-Disruption-Events\env\lib\site-packages\pandas\core\series.py", line 1037, in __getitem__
    return self._values[key]
IndexError: index -1 is out of bounds for axis 0 with size 0


--------------------------------------------------------------------------------
assistant (to user_proxy):

I apologize for the confusion. The error message in

KeyboardInterrupt: 

In [ ]:
from typing import Dict, Union
from IPython import get_ipython

class IPythonUserProxyAgent(autogen.UserProxyAgent):
    def __init__(self, name: str, **kwargs):
        super().__init__(name, **kwargs)
        self._ipython = get_ipython()

    def generate_init_message(self, *args, **kwargs) -> Union[str, Dict]:
        return super().generate_init_message(*args, **kwargs) + """
If you suggest code, the code will be executed in IPython."""

    def run_code(self, code, **kwargs):
        result = self._ipython.run_cell("%%capture --no-display cap\n" + code)
        log = self._ipython.ev("cap.stdout")
        log += self._ipython.ev("cap.stderr")
        if result.result is not None:
            log += str(result.result)
        exitcode = 0 if result.success else 1
        if result.error_before_exec is not None:
            log += f"\n{result.error_before_exec}"
            exitcode = 1
        if result.error_in_exec is not None:
            log += f"\n{result.error_in_exec}"
            exitcode = 1
        return exitcode, log, None